In [1]:
data Exp = Val Int
         | Add Exp Exp
         | Sub Exp Exp
         | Mul Exp Exp
         | Div Exp Exp
         deriving (Show)

In [2]:
eval :: Exp -> Int
eval (Val n) = n
eval (Add e1 e2) = eval e1 + eval e2
eval (Sub e1 e2) = eval e1 - eval e2
eval (Mul e1 e2) = eval e1 * eval e2
eval (Div e1 e2) = eval e1 `div` eval e2

In [3]:
eval (Add (Val 3) (Val 4) `Mul` Val 5)

35

In [4]:
eval (Div (Val 3) (Val 0)) -- 이런 경우는 Nothing을 처리하자

: 

```haskell
data Maybe a = Nothing | Just a 
```

In [5]:
{-
instance Monad Maybe where
  Nothing >>= _ = Nothing
  Just v  >>= f = f v

  return = Just
-}

evm :: Exp -> Maybe Int
evm (Val n) = return n
evm (Add e1 e2) = do v1 <- evm e1
                     v2 <- evm e2
                     return (v1+v2)
evm (Sub e1 e2) = do v1 <- evm e1
                     v2 <- evm e2
                     return (v1-v2)
evm (Mul e1 e2) = do v1 <- evm e1
                     v2 <- evm e2
                     return (v1*v2)
evm (Div e1 e2) = do v1 <- evm e1
                     v2 <- evm e2
                     if v2==0 then Nothing
                              else return (v1*v2)

{-
evm :: Exp -> Maybe Int
evm (Val n) = return n
evm (Add e1 e2) = evm e1 >>= \v1 ->
                  evm e2 >>= \v2 ->
                  return (v1+v2)
evm (Sub e1 e2) = evm e1 >>= \v1 ->
                  evm e2 >>= \v2 ->
                  return (v1-v2)
evm (Mul e1 e2) = evm e1 >>= \v1 ->
                  evm e2 >>= \v2 ->
                  return (v1*v2)
evm (Div e1 e2) = evm e1 >>= \v1 ->
                  evm e2 >>= \v2 ->
                  if v2==0 then Nothing
                           else return (v1*v2)
-}

In [6]:
evm (Add (Val 3) (Val 4) `Mul` Val 5)

Just 35

In [7]:
evm (Div (Val 3) (Val 0))

Nothing

In [8]:
evM :: Exp -> Maybe Int
evM (Val n) = Just n
evM (Add e1 e2) =
  case m1 of
    Nothing -> Nothing
    Just v1 -> case m2 of
                 Nothing -> Nothing
                 Just v2 -> Just(v1 + v2)
  where m1 = evM e1
        m2 = evM e2
evM (Sub e1 e2) =
  case m1 of
    Nothing -> Nothing
    Just v1 -> case m2 of
                 Nothing -> Nothing
                 Just v2 -> Just(v1 - v2)
  where m1 = evM e1
        m2 = evM e2
evM (Mul e1 e2) =
  case m1 of
    Nothing -> Nothing
    Just v1 -> case m2 of
                 Nothing -> Nothing
                 Just v2 -> Just(v1 * v2)
  where m1 = evM e1
        m2 = evM e2
evM (Div e1 e2) =
  case m1 of
    Nothing -> Nothing
    Just v1 -> case m2 of
                 Nothing -> Nothing
                 Just v2 -> if v2==0 then Nothing
                                     else Just(v1 `div` v2)
  where m1 = evM e1
        m2 = evM e2

In [9]:
evM (Add (Val 3) (Val 4) `Mul` Val 5)

Just 35

In [10]:
evM (Div (Val 3) (Val 0))

Nothing

```haskell
data Either a b = Left a | Right b 
```

In [14]:
{- (>>=) :: Either String Int
      -> (Int -> Either String Int)
      -> Either String Int
Left er >>= _  = Left er
Right v >>= f  = f v

return :: Int -> Either String Int
return = Right
-}

eve :: Exp -> Either String Int
eve (Val n) = return n
eve (Add e1 e2) = do v1 <- eve e1
                     v2 <- eve e2
                     return (v1+v2)
eve (Sub e1 e2) = do v1 <- eve e1
                     v2 <- eve e2
                     return (v1-v2)
eve (Mul e1 e2) = do v1 <- eve e1
                     v2 <- eve e2
                     return (v1*v2)
eve (Div e1 e2) = do v1 <- eve e1
                     v2 <- eve e2
                     if v2==0 then Left "Div by 0"
                              else return (v1*v2)

In [15]:
eve (Add (Val 3) (Val 4) `Mul` Val 5)

Right 35

In [16]:
eve (Div (Val 3) (Val 0))

Left "Div by 0"

In [12]:
evE :: Exp -> Either String Int
evE (Val n) = Right n
evE (Add e1 e2) =
  case m1 of
    Left _   -> m1
    Right v1 -> case m2 of
                 Left _ -> m2
                 Right v2 -> Right(v1 + v2)
  where m1 = evE e1
        m2 = evE e2
evE (Sub e1 e2) =
  case m1 of
    Left _ -> m1
    Right v1 -> case m2 of
                 Left _ -> m2
                 Right v2 -> Right(v1 - v2)
  where m1 = evE e1
        m2 = evE e2
evE (Mul e1 e2) =
  case m1 of
    Left _ -> m1
    Right v1 -> case m2 of
                 Left _ -> m2
                 Right v2 -> Right(v1 * v2)
  where m1 = evE e1
        m2 = evE e2
evE (Div e1 e2) =
  case m1 of
    Left _ -> m1
    Right v1 -> case m2 of
                 Left _ -> m2
                 Right v2 -> if v2==0 then Left "Div by 0"
                                      else Right(v1 `div` v2)
  where m1 = evE e1
        m2 = evE e2

여러가지 계산결과의 가능성을 시뮬레이션하는 모나드를 리스트로

In [20]:
graph = [('a','b'),('a','d')
        ,('b','c')
        ,('c','d')
        ,('d','e')
        ,('e','c')]

step :: Char -> [Char]
step x = [y1 | (x1,y1)<-graph, x==x1]

In [28]:
path3 :: Char -> [Char]
path3 x = do x1 <- step x
             x2 <- step x1
             x3 <- step x2
             return x3

path :: Int -> Char -> [Char]
path 0 x = return x
path n x = do y <- step x
              path (n-1) y

In [29]:
path3 'a'

"dc"

In [34]:
path 0 'a'
path 1 'a'
path 2 'a'
path 3 'a'

"a"

"bd"

"ce"

"dc"